# Age-Metallicity Relation

In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from scipy.stats import binned_statistic
from scipy.stats import gaussian_kde
from scipy.stats import median_abs_deviation as mad
import pandas as pd
from tqdm import tqdm

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse
from auriga.support import make_snapshot_number

In [3]:
figure_setup()

In [4]:
settings = Settings()

In [5]:
N_RADIAL_BINS = 14
N_BINS_MAP = 200
MAP_VMIN, MAP_VMAX = 1E0, 1E3
AGE_RANGE = (0, 14)
SAMPLE = settings.groups["Included"]

In [6]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [13]:
def read_data(simulation: str) -> pd.DataFrame:
    """
    This method returns data related to the analysis in this notebook.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    pd.DataFrame
        The properties in a Pandas DataFrame.
    """
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_stellar_age()
    s.add_metal_abundance(of="Fe", to='H')
    s.add_circularity()
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {
        "StellarAge_Gyr": s.stellar_age[is_real_star & is_main_obj],
        "[Fe/H]": s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
        "xPosition_ckpc": s.pos[is_real_star & is_main_obj, 0],
        "yPosition_ckpc": s.pos[is_real_star & is_main_obj, 1],
        "zPosition_ckpc": s.pos[is_real_star & is_main_obj, 2],
        "ComponentTag": s.region_tag[is_real_star & is_main_obj]}

    df = pd.DataFrame(props)
    df[~np.isfinite(df)] = np.nan
    df.dropna(inplace=True)
    return df

In [8]:
def get_stats_of_sample(simulations: list) -> pd.DataFrame:
    """
    This method returns the data of interest over the selected sample.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    pd.DataFrame
        The properties in a Pandas DataFrame.
    """

    # Accumulate data for all galaxies
    ages = list()
    abundances = list()
    component_tags = list()

    for simulation in simulations:
        df = read_data(simulation, include_region=True)

        ages += list(df["StellarAge_Gyr"].to_numpy())
        abundances += list(df["[Fe/H]"].to_numpy())
        component_tags += list(df["ComponentTag"].to_numpy())

    ages = np.array(ages)
    abundances = np.array(abundances)
    component_tags = np.array(component_tags)

    median, bin_edges, _ = binned_statistic(
        x=ages,
        values=abundances,
        statistic=np.nanmedian,
        bins=N_RADIAL_BINS,
        range=AGE_RANGE
    )
    bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2

    stats = {"Median[Fe/H]": median,
             "AgeBinCenters_Gyr": bin_centers}
    
    # Add median por each component
    for i in range(len(settings.components)):
        median, _, _ = binned_statistic(
            x=ages[component_tags == i],
            values=abundances[component_tags == i],
            statistic=np.nanmedian,
            bins=N_RADIAL_BINS,
            range=AGE_RANGE)
        stats[f"Median[Fe/H]_{settings.components[i]}"] = median

    stats = pd.DataFrame(stats)
    stats.to_csv(f"../results/age_metallicity_sample_median{SUFFIX}")
    return stats

### Median Age-Metallicity Relation Statistics for Sample

In [9]:
try:
    stats = pd.read_csv(
        f"../results/age_metallicity_sample_median{SUFFIX}.csv",
        index_col=0)
except FileNotFoundError:
    stats = get_stats_of_sample([f"au{i}_or_l4_s127" for i in SAMPLE])

### AMR for Sample

In [33]:
fig = plt.figure(figsize=(7, 8))
gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.tick_params(which='both', direction="in")
    if ax == axs[-1, -1]: ax.axis("off")
    ax.set_xlim(AGE_RANGE)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_ylim(-4, 3)
    ax.set_yticks([-3, -2, -1, 0, 1, 2])
    ax.set_axisbelow(True)
    if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
        ax.set_xlabel("Age [Gyr]")
        ax.tick_params(labelbottom=True)
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel("[Fe/H]")
    if ax == axs[-1, -1]:
        ax.scatter(1, 0, c="white", s=30, lw=1, marker="o", ec='k', zorder=15)
        ax.text(x=2, y=0, size=8.0, ha="left", va="center",
                s=r"$\textbf{All}$", c='k')
        for i in range(len(settings.components)):
            color = list(settings.component_colors.values())[i]
            marker = list(settings.component_markers.values())[i]
            ax.scatter(1, 0 - 0.85 * (i + 1), c=color, s=30, lw=1, marker=marker,
                       ec='k', zorder=15)
            ax.text(
                x=2, y=0 - 0.85 * (i + 1), size=8.0, ha="left", va="center",
                s=r"$\textbf{" + list(settings.component_labels.values())[i] \
                    + "}$",
                c=list(settings.component_colors.values())[i])

for i in range(len(settings.groups["Included"])):
    ax = axs.flatten()[i]
    galaxy = settings.groups["Included"][i]
    simulation = f"au{galaxy}_or_l4_s127"
    label = f"Au{galaxy}"
    df = read_data(simulation=simulation)
    ax.hist2d(df["StellarAge_Gyr"],
              df["[Fe/H]"],
              cmap='nipy_spectral',
              bins=N_BINS_MAP,
              range=[ax.get_xlim(), ax.get_ylim()],
              norm=mcolors.LogNorm(vmin=MAP_VMIN, vmax=MAP_VMAX),
              rasterized=True)

    # Plot the median for this galaxy
    stat, bin_edges, _ = binned_statistic(
        x=df["StellarAge_Gyr"],
        values=df["[Fe/H]"],
        statistic=np.nanmedian,
        bins=N_RADIAL_BINS,
        range=AGE_RANGE)
    bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
    ax.plot(bin_centers, stat)

    # Calculate the median for each component
    medians = [(np.nanmedian(df["StellarAge_Gyr"]), np.nanmean(df["[Fe/H]"]))]
    ax.scatter(*medians[0], c="white", s=30, lw=1, marker="o", ec='k',
               zorder=15)
    for i in range(len(settings.components)):
        is_region = (df["ComponentTag"] == i)
        medians.append((np.nanmedian(df["StellarAge_Gyr"][is_region]),
                        np.nanmedian(df["[Fe/H]"][is_region])))
        color = list(settings.component_colors.values())[i]
        marker = list(settings.component_markers.values())[i]
        ax.scatter(*medians[i + 1], c=color, s=30, lw=1, edgecolors='k',
                   zorder=15, marker=marker)

    ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
            size=6.0, transform=ax.transAxes, ha='left', va='top')

    # Plot the median for the sample and show the MSE
    ax.plot(stats["AgeBinCenters_Gyr"], stats["Median[Fe/H]"], c='k', ls="--")
    mse = ((stat - stats["Median[Fe/H]"])**2).sum() / stat.shape[0]
    ax.text(x=0.05, y=0.05,
            s=r"$\mathrm{MSE} = " + str(np.round(mse, 3)).ljust(5, '0') + "$",
            size=6.0, transform=ax.transAxes, ha='left', va='bottom')

    fig.savefig(f"../images/age_metallicity_by_region/included{SUFFIX}.pdf")


plt.close(fig)

### AMR for Galaxy

In [36]:
SIMULATION = "au6_or_l4_s127"
galaxy, _, _, _ = parse(SIMULATION)

In [15]:
df = read_data(simulation=SIMULATION, include_region=True)

In [16]:
medians = [(np.nanmedian(df["StellarAge_Gyr"]), np.nanmean(df["[Fe/H]"]))]
for i in range(len(settings.components)):
    is_region = (df["ComponentTag"] == i)
    medians.append((np.nanmedian(df["StellarAge_Gyr"][is_region]),
                    np.nanmedian(df["[Fe/H]"][is_region])))

In [65]:
fig = plt.figure(figsize=(7.4, 2.0))
gs = fig.add_gridspec(nrows=1, ncols=5, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flat:
    ax.set_xlim(0, 14)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_xlabel("Stellar Age [Gyr]")

    ax.set_ylim(-4, 3)
    ax.set_yticks([-3, -2, -1, 0, 1, 2])
    ax.set_ylabel("[Fe/H]")

    ax.tick_params(which='both', direction="in")
    ax.label_outer()

axs[0].hist2d(df["StellarAge_Gyr"], df["[Fe/H]"],
              cmap='nipy_spectral', bins=N_BINS_MAP,
              range=[axs[0].get_xlim(), axs[0].get_ylim()],
              norm=mcolors.LogNorm(vmin=MAP_VMIN, vmax=MAP_VMAX),
              rasterized=True)[-1]
axs[0].plot(axs[0].get_xlim(), [medians[0][1]] * 2, ls=":", c='k')
axs[0].plot([medians[0][0]] * 2, axs[0].get_ylim(), ls=":", c='k')
axs[0].scatter(*medians[0], c="white", s=30, lw=1, marker="o", ec='k',
               zorder=15)
axs[0].text(x=0.05, y=0.05, size=8.0, ha="left", va="bottom",
            s=r"$\textbf{All}$", c='k', transform=axs[0].transAxes)
stat, bin_edges, _ = binned_statistic(
    x=df["StellarAge_Gyr"], values=df["[Fe/H]"],
    statistic=np.nanmedian, bins=N_RADIAL_BINS, range=AGE_RANGE)
bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
axs[0].plot(bin_centers, stat, color='k', lw=1.75)
axs[0].plot(bin_centers, stat, color="white", lw=1)
axs[0].plot(stats["AgeBinCenters_Gyr"], stats[f"Median[Fe/H]"],
            c="black", ls="--", lw=2.0)
mse = ((stat - stats[f"Median[Fe/H]"])**2).sum() / stat.shape[0]
axs[0].text(
    x=0.05, y=0.95,
    s="$\mathrm{MSE} = " + str(np.round(mse, 3)).ljust(5, '0') + "$",
    size=8.0, transform=axs[0].transAxes, ha="left", va="top",
    )


for i in range(len(settings.components)):
    is_region = (df["ComponentTag"] == i)
    ax = axs[i + 1]
    color = list(settings.component_colors.values())[i]
    marker = list(settings.component_markers.values())[i]

    im = ax.hist2d(
        df["StellarAge_Gyr"][is_region], df["[Fe/H]"][is_region],
        cmap='nipy_spectral', bins=N_BINS_MAP,
        range=[ax.get_xlim(), ax.get_ylim()],
        norm=mcolors.LogNorm(vmin=MAP_VMIN, vmax=MAP_VMAX),
        rasterized=True)[-1]

    ax.text(
        x=0.05, y=0.05, size=8.0, ha="left", va="bottom",
        s=r"$\textbf{" + list(settings.component_labels.values())[i] + "}$",
        c=list(settings.component_colors.values())[i],
        transform=ax.transAxes)

    ax.plot(ax.get_xlim(), [medians[i + 1][1]] * 2, ls=":", c=color)
    ax.plot([medians[i + 1][0]] * 2, ax.get_ylim(), ls=":", c=color)
    ax.scatter(*medians[i + 1], c=color, s=30, lw=1, edgecolors='k',
               zorder=15, marker=marker)

    # Median for this galaxy
    stat, bin_edges, _ = binned_statistic(
        x=df["StellarAge_Gyr"][is_region],
        values=df["[Fe/H]"][is_region],
        statistic=np.nanmedian,
        bins=N_RADIAL_BINS,
        range=AGE_RANGE)
    bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
    ax.plot(bin_centers, stat, color='k', lw=1.75)
    ax.plot(bin_centers, stat, color=color, lw=1)

    # Median for the sample
    ax.plot(stats["AgeBinCenters_Gyr"],
            stats[f"Median[Fe/H]_{settings.components[i]}"],
            c="black", ls="--", lw=2.0)

    # Calculate and write MSE
    mse = ((stat - stats[f"Median[Fe/H]_{settings.components[i]}"])**2).sum() \
        / stat.shape[0]
    ax.text(
        x=0.05, y=0.95,
        s="$\mathrm{MSE} = " + str(np.round(mse, 3)).ljust(5, '0') + "$",
        size=8.0, transform=ax.transAxes, ha="left", va="top",
        )

# cbar = fig.colorbar(im, ax=axs[-1], orientation='vertical',
#                     label=r'$N_\mathrm{stars}$', pad=0)
# cbar.ax.set_yticks([1E0, 1E1, 1E2, 1E3])

axs[0].text(x=0.95, y=0.95, s=r"$\texttt{" + f"Au{galaxy}" + "}$",
            size=8.0, ha="right", va="top", transform=axs[0].transAxes)

fig.savefig(f"../images/age_metallicity_by_region/{SIMULATION}{SUFFIX}.pdf")
plt.close(fig)

### Median Scatter

In [34]:
components = []
median_abundances = []
median_ages = []

for simulation in tqdm([f"au{i}_or_l4_s127" for i in SAMPLE]):
    df = read_data(simulation=simulation)
    for i in range(len(settings.components)):
        is_region = (df["ComponentTag"] == i)
        components.append(i)
        median_abundances.append(np.nanmedian(df["[Fe/H]"][is_region]))
        median_ages.append(np.nanmedian(df["StellarAge_Gyr"][is_region]))

components = np.array(components)
median_abundances = np.array(median_abundances)
median_ages = np.array(median_ages)

  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [43:36<00:00, 136.78s/it]


In [36]:
markers = list(settings.component_markers.values())
colors = list(settings.component_colors.values())
labels = list(settings.component_labels.values())

fig, axs = plt.subplots(
    figsize=(7.4, 2.0), nrows=1, ncols=4, sharey=True, sharex=True,
    gridspec_kw={"hspace": 0.0, "wspace": 0.0})

for ax in axs.flat:
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.tick_params(which='both', direction="in")

    ax.set_xlim(0, 14)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_xlabel("median(Age) [Gyr]")

    ax.set_ylim(-1.0, 0.7)
    ax.set_ylabel('median([Fe/H])')
    ax.set_axisbelow(True)

    ax.label_outer()

for i, ax in enumerate(axs.flatten()):
    for j in range(4):
        label = labels[j] if j == i else None
        color = colors[j] if j == i else "silver"
        zorder = 10 if j == i else 5
        ax.scatter(
            median_ages[components == j],
            median_abundances[components == j],
            c=color, label=label, zorder=zorder,
            s=20, linewidths=0.4, edgecolors="white", marker=markers[j])

        if j == i:
            age_pdf = gaussian_kde(median_ages[components == j])
            pdf_x = np.linspace(0, 14, 100)
            pdf_y = age_pdf(pdf_x) / np.max(age_pdf(pdf_x)) * 0.2 - 1.0
            ax.fill_between(
                x=pdf_x, y1=-1.0, y2=pdf_y, edgecolor=color,
                facecolor=mcolors.TABLEAU_COLORS[color] + "30")
            
            abundance_pdf = gaussian_kde(median_abundances[components == j])
            pdf_y = np.linspace(-1.0, 0.7, 100)
            pdf_x = abundance_pdf(pdf_y) / np.max(abundance_pdf(pdf_y)) * 2
            ax.fill_betweenx(
                y=pdf_y, x1=-pdf_x + 14, x2=14, edgecolor=color,
                facecolor=mcolors.TABLEAU_COLORS[color] + "30")

    ax.legend(loc="upper left", framealpha=0, fontsize=7.5)

fig.savefig(
    f"../images/age_metallicity_by_region/included_scatter{SUFFIX}.pdf")
plt.close(fig)